# Combine 

In [2]:
import os
import pandas as pd

def extract_feature_name_from_path(path):
    """Guess feature name from filename (e.g. '1.Số lượng trâu 1-1.xlsx' -> 'Buffalo_Count')"""
    fname = os.path.basename(path)
    name = fname.replace(".xlsx", "")
    return name.replace(" ", "_").replace(".", "_")

def load_and_rename(file_path):
    """Load Excel file and convert wide to long format."""
    df = pd.read_excel(file_path)
    df = df.melt(id_vars=["Location"], var_name="Year", value_name="Value")
    df["Year"] = df["Year"].astype(int)
    feature_name = extract_feature_name_from_path(file_path)
    df = df.rename(columns={"Value": feature_name})
    return df

def merge_all_files(folder):
    """Walk through a folder, load all .xlsx files, and merge by Location-Year."""
    merged_df = None
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith(".xlsx"):
                full_path = os.path.join(root, file)
                df = load_and_rename(full_path)
                if merged_df is None:
                    merged_df = df
                else:
                    merged_df = pd.merge(merged_df, df, on=["Location", "Year"], how="outer")
    return merged_df

if __name__ == "__main__":
    folder_path = "[Carbon Farming] Dữ liệu NN Thanh Hóa"
    merged = merge_all_files(folder_path)
    merged = merged.sort_values(by=["Location", "Year"]).reset_index(drop=True)
    merged.to_csv("dataset/merged_dataset.csv", index=False)
    print("✅ Merged dataset saved to data/merged_dataset.csv")

✅ Merged dataset saved to data/merged_dataset.csv


# Preprocessing

In [3]:
import pandas as pd

# Load merged dataset
merged_df = pd.read_csv("dataset/merged_dataset.csv")

# Show basic info
print("\n[INFO] Shape:", merged_df.shape)
print("\n[INFO] Columns:", merged_df.columns.tolist())

# Show nulls
print("\n[INFO] Missing values:")
print(merged_df.isnull().sum())

# Show sample rows
print("\n[INFO] Sample data:")
print(merged_df.head())



[INFO] Shape: (210, 15)

[INFO] Columns: ['Location', 'Year', '5_Diện_tích_cây_cao_su', '1_Số_lượng_trâu_1-1', '2_Số_lượng_bò_1-1', '3_Số_lượng_lợn_1-1', '4_Số_lượng_dê_1-1', '5_Số_lượng_gia_cầm_1-1', '1_Diện_tích_lúa_đông_xuân', '2_Diện_tích_lúa_vụ_mùa', '3_Năng_suất_vụ_Đông_Xuân', '4_Năng_suất_vụ_mùa', '5_Sản_lượng_lúa_Đông_Xuân', '6_Sản_lượng_lúa_vụ_Mùa', '1_Diện_tích_rừng']

[INFO] Missing values:
Location                      0
Year                          0
5_Diện_tích_cây_cao_su       75
1_Số_lượng_trâu_1-1          75
2_Số_lượng_bò_1-1            75
3_Số_lượng_lợn_1-1           75
4_Số_lượng_dê_1-1            75
5_Số_lượng_gia_cầm_1-1       75
1_Diện_tích_lúa_đông_xuân    21
2_Diện_tích_lúa_vụ_mùa       21
3_Năng_suất_vụ_Đông_Xuân     21
4_Năng_suất_vụ_mùa           21
5_Sản_lượng_lúa_Đông_Xuân    21
6_Sản_lượng_lúa_vụ_Mùa       21
1_Diện_tích_rừng             75
dtype: int64

[INFO] Sample data:
         Location  Year  5_Diện_tích_cây_cao_su  1_Số_lượng_trâu_1-1  \
0  Huyện

In [6]:
import pandas as pd

THRESHOLD = 0.6  # If more than 50% of features are missing in a row, drop it

def clean_dataset(input_path, output_path):
    df = pd.read_csv(input_path)

    # Count how many features (excluding Location, Year)
    feature_cols = [col for col in df.columns if col not in ["Location", "Year"]]
    feature_count = len(feature_cols)

    # Step 1: drop rows with too many missing values
    df["missing_count"] = df[feature_cols].isnull().sum(axis=1)
    df = df[df["missing_count"] <= (feature_count * THRESHOLD)].drop(columns=["missing_count"])

    # Step 2: fill missing values by mean of that Location across other years
    df[feature_cols] = df.groupby("Location")[feature_cols].transform(lambda x: x.fillna(x.mean()))

    # Save cleaned file
    df.to_csv(output_path, index=False)
    print(f"✅ Cleaned dataset saved to {output_path}")

if __name__ == "__main__":
    clean_dataset("dataset/merged_dataset.csv", "dataset/cleaned_dataset.csv")


✅ Cleaned dataset saved to dataset/cleaned_dataset.csv
